## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
import requests
from bs4 import BeautifulSoup
from shapely.geometry import Point

## Read-in

In [42]:
df = pd.read_csv('LISF_July_August_2024.csv',dtype='str')

In [43]:
df['Doc Recorded'].isna().value_counts()

Doc Recorded
False    2388
Name: count, dtype: int64

In [44]:
# Define the regex pattern to split the text
pattern = r'(\d+-\d+-\d+-\d+-\d+)\s(.*)'

# Apply regex and split the text into two columns
df[['PIN', 'Address']] = df['1st PIN'].str.extract(pattern)

# Remove leading/trailing whitespace from the address column
df['Address'] = df['Address'].str.strip()

In [45]:
df = df.drop(columns=['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0','1st PIN'])

## Clean, drop, and convert

In [46]:
df['Doc Recorded'] = pd.to_datetime(df['Doc Recorded'], format='mixed')

In [47]:
df = df.drop_duplicates()

In [48]:
len(df)

1888

## Data Stuff

In [49]:
df['1st Grantor'] = df['1st Grantor'].fillna('NA')
df['1st Grantee'] = df['1st Grantee'].fillna('NA')

In [50]:
municipal_authority = ['city', 'town', 'municipality', 'village','transit auth','department of transp']  # list of municipal authority keywords
df = df[~df['1st Grantor'].str.contains('|'.join(municipal_authority), case=False) & 
        ~df['1st Grantee'].str.contains('|'.join(municipal_authority), case=False)]

In [52]:
df['COLOR'] = df['Doc Recorded'].apply(lambda x: 'red' if x.month == 7 else 'yellow')

## Get mortgage amounts

In [53]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'TE': 'Trailers'
}

In [54]:
def mortgage_url_snagger(URL, headers):
    response = requests.get(URL, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    for link in soup.find_all('a', href=True):
        if link['href'].startswith('/Document/Detail'):
            mortgage_url = 'https://crs.cookcountyclerkil.gov' + link['href']
            return mortgage_url

In [55]:
def mortgage_consi_snagger(URL, headers):
    response = requests.get(URL, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Find all tr tags
    trs = soup.find_all('tr')

    # Loop through each tr tag and look for the td tag containing 'Consideration Amount' label
    for tr in trs:
        td = tr.find('th', text='Consideration Amount:')
        if td:
            # If the td tag is found, get the next td tag containing the amount
            amount_td = td.find_next_sibling('td')
            if amount_td:
                # Print the amount
                amount = amount_td.text.strip()
                return amount
            else:
                return 'not found'

In [56]:
df['mortgage_urls'] = df['deed_urls'].apply(lambda x: mortgage_url_snagger(x, headers))

In [57]:
df['mortgage_amount'] = df['mortgage_urls'].apply(lambda x: mortgage_consi_snagger(x, headers) if x is not None else None)

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_24835/2390556658.py:9: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  td = tr.find('th', text='Consideration Amount:')


In [58]:
df[['mortgage_urls','mortgage_amount']] = df[['mortgage_urls','mortgage_amount']].fillna('NA')

## Geocode

In [59]:
len(df)

1494

In [60]:
df['geo_address'] = df['Address'] + ' Cook County, IL'

In [61]:
len(df)

1494

In [62]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [63]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'geo_address' column and store the results in 'geocoded' column
df['geocoded'] = df['geo_address'].apply(geocode)

In [64]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x == 'None' else x.strip('()').split(', ', 1)).apply(pd.Series)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## HTML Popup Formatter

In [65]:
df.columns

Index(['View Doc', 'Doc Number', 'Doc Recorded', 'Doc Executed', 'Doc Type',
       'Consi. Amt.', '1st Grantor', '1st Grantee', 'Assoc. Doc#', 'deed_urls',
       'PIN', 'Address', 'COLOR', 'mortgage_urls', 'mortgage_amount',
       'geo_address', 'geocoded', 'lat', 'lon'],
      dtype='object')

In [66]:
def popup_html(row):
    grantor = row['1st Grantor']
    grantee = row['1st Grantee']
    PIN = row['PIN']
    Address = row['Address']
    mortgage_amount = row['mortgage_amount']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Lender: </strong>{}'''.format(grantor) + '''<br>
    <strong>Borrower: </strong>{}'''.format(grantee) + '''<br>
    <strong>PIN: </strong>{}'''.format(PIN) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>Mortgage Amount: </strong>{}'''.format(mortgage_amount) + '''<br>
    </html>
    '''
    return html

In [67]:
df['COLOR'].value_counts()

COLOR
red       795
yellow    699
Name: count, dtype: int64

In [71]:
# df.to_csv("backup_copy.csv")

In [32]:
# df['COLOR'] = df['COLOR'].replace('', 'yellow')

In [68]:
import folium
from folium.plugins import MarkerCluster
import numpy as np

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Cook County Pending Foreclosures')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'July and August')

### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=9.5, tiles=None)

# Create two FeatureGroups for different color pins
fg_red = folium.FeatureGroup(name=f'May') ## 
fg_yellow = folium.FeatureGroup(name='June')

for index, row in df.iterrows():
    lat = row['lat']
    lon = row['lon']
    color = row['COLOR']
    if pd.notnull(lat) and pd.notnull(lon) and color == 'red':
        marker = folium.CircleMarker(
            location=[lat, lon],
            radius=5,
            fill=True,
            color=color,
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_red)
    elif pd.notnull(lat) and pd.notnull(lon) and color == 'yellow':
        marker = folium.CircleMarker(
            location=[lat, lon],
            radius=5,
            fill=True,
            color=color,
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_yellow)
    else:
        continue

# Add the FeatureGroups to the map
fg_red.add_to(m)
fg_yellow.add_to(m)

folium.TileLayer('OpenStreetMap', control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
folium.TileLayer('CartoDBpositron', control=False).add_to(m) 
            
# Display map
m

In [69]:
m.save('index.html')

## Map URL Snagger

In [70]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/lis_pendens_scraper_july_august_2024


## Stats for story

In [71]:
df['mortgage_amount_int'] = df['mortgage_amount'].str.replace('$','')
df['mortgage_amount_int'] = df['mortgage_amount_int'].str.replace(',','')
df = df.loc[df['mortgage_amount_int'] != 'NA']
df['mortgage_amount_int'] = df['mortgage_amount_int'].astype(float).fillna(0).astype(int)

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_24835/159485817.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mortgage_amount_int'] = df['mortgage_amount_int'].astype(float).fillna(0).astype(int)


In [75]:
# Count number of first month dates
dates_count = len(df.loc[df['Doc Recorded'].dt.month == 7])

print(f'Number of July dates: {dates_count}')

Number of July dates: 731


In [77]:
# Count number of first month dates
dates_count_august = len(df.loc[df['Doc Recorded'].dt.month == 8])

print(f'Number of August dates: {dates_count_august}')

Number of August dates: 645


In [78]:
df['Doc Recorded'].dt.month.value_counts()

Doc Recorded
7    731
8    645
Name: count, dtype: int64

In [79]:
# Filter for March
df_july = df[df['Doc Recorded'].dt.month == 7]

# Filter for April
df_august = df[df['Doc Recorded'].dt.month == 8]

In [80]:
df_july.reset_index(inplace=True, drop=True)
df_august.reset_index(inplace=True, drop=True)

In [36]:
# x.at[123,'mortgage_amount_int']=33130000
# x.at[123,'mortgage_amount']='$33,130,000'

# x.at[16,'mortgage_amount_int']=173600
# x.at[16,'mortgage_amount']='$173,600'

In [81]:
print(f'July foreclosures: {len(df_july)}')
print('----------')
print(f'August foreclosures: {len(df_august)}')

July foreclosures: 731
----------
August foreclosures: 645


In [82]:
df_july.sort_values(by='mortgage_amount_int',ascending=False).head(2)

,View Doc,Doc Number,Doc Recorded,Doc Executed,Doc Type,Consi. Amt.,1st Grantor,1st Grantee,Assoc. Doc#,deed_urls,PIN,Address,COLOR,mortgage_urls,mortgage_amount,geo_address,geocoded,lat,lon,mortgage_amount_int
152,View,2420618107,2024-07-24,7/18/2024,LIS PENDENS FORECLOSURE,NaN,WELLS FARGO TRUST COMPANY NATIONAL ASSN TR,TERRA FUNDINGBURNHAM CTR LLC,1307722130.0,https://crs.cookcountyclerkil.gov/Document/Det...,17-09-459-002-0000,"111 W WASHINGTON ST, CHICAGO",red,https://crs.cookcountyclerkil.gov/Document/Det...,"$42,000,000.00","111 W WASHINGTON ST, CHICAGO Cook County, IL","(41.8830402, -87.6314065)",41.883040,-87.631406,42000000
7,View,2421311021,2024-07-31,7/30/2024,LIS PENDENS FORECLOSURE,NaN,REPUBLIC BK OF CHICAGO,1454 S MICHIGAN LLC,2236422010.0,https://crs.cookcountyclerkil.gov/Document/Det...,17-22-107-066-0000,"1464 S MICHIGAN AVE, CHICAGO",red,https://crs.cookcountyclerkil.gov/Document/Det...,"$11,850,000.00","1464 S MICHIGAN AVE, CHICAGO Cook County, IL","(41.86225779999999, -87.62428899999999)",41.862258,-87.624289,11850000


In [83]:
df_august.sort_values(by='mortgage_amount_int',ascending=False).head(2)

,View Doc,Doc Number,Doc Recorded,Doc Executed,Doc Type,Consi. Amt.,1st Grantor,1st Grantee,Assoc. Doc#,deed_urls,PIN,Address,COLOR,mortgage_urls,mortgage_amount,geo_address,geocoded,lat,lon,mortgage_amount_int
282,View,2421924711,2024-08-06,8/6/2024,LIS PENDENS FORECLOSURE,NaN,FIRST EAGLE BANK,MERCHANTS BK OF INDIANA,1935141009.0,https://crs.cookcountyclerkil.gov/Document/Det...,14-33-422-016-0000,"1639 N NORTH PARK AVE, CHICAGO",yellow,https://crs.cookcountyclerkil.gov/Document/Det...,"$3,000,000.00","1639 N NORTH PARK AVE, CHICAGO Cook County, IL","(41.9121939, -87.6363868)",41.912194,-87.636387,3000000
222,View,2423609047,2024-08-23,NaN,LIS PENDENS FORECLOSURE,NaN,CNB BK & TRUST NA,ALEXI CORPORATION,2222328282.0,https://crs.cookcountyclerkil.gov/Document/Det...,31-20-302-037-0000,"7155 LINCOLN HWY, MATTESON",yellow,https://crs.cookcountyclerkil.gov/Document/Det...,"$2,800,000.00","7155 LINCOLN HWY, MATTESON Cook County, IL","(41.5061598, -87.7897861)",41.506160,-87.789786,2800000


In [105]:
print(df_july.iloc[152]['mortgage_urls'])

https://crs.cookcountyclerkil.gov/Document/Detail?dId=MzI0NDQzMDg1&hId=ZWU3YjZlNDAzMmZlYjM4MjkwZmRlZWFiOWM4YzI1NTg4MjY1NmNjMDUxOWE1MTM2ZDVlMjVkNzA1YzkyYzM5Nw2


In [87]:
print(df_august.iloc[282]['mortgage_urls'])

https://crs.cookcountyclerkil.gov/Document/Detail?dId=Mzc2ODA3ODg1&hId=MDIxOTAyNzU5Njk2Mjk3ODkyOWU1YjcyNTczNDVkODE5YmIyZDFjNjlhYTg0Y2RjMmQxNDMyZDk2NDY2MjQ4Yg2


In [90]:
df_august.sort_values(by='mortgage_amount_int',ascending=False).iloc[0:1]

,View Doc,Doc Number,Doc Recorded,Doc Executed,Doc Type,Consi. Amt.,1st Grantor,1st Grantee,Assoc. Doc#,deed_urls,PIN,Address,COLOR,mortgage_urls,mortgage_amount,geo_address,geocoded,lat,lon,mortgage_amount_int
282,View,2421924711,2024-08-06,8/6/2024,LIS PENDENS FORECLOSURE,NaN,FIRST EAGLE BANK,MERCHANTS BK OF INDIANA,1935141009.0,https://crs.cookcountyclerkil.gov/Document/Det...,14-33-422-016-0000,"1639 N NORTH PARK AVE, CHICAGO",yellow,https://crs.cookcountyclerkil.gov/Document/Det...,"$3,000,000.00","1639 N NORTH PARK AVE, CHICAGO Cook County, IL","(41.9121939, -87.6363868)",41.912194,-87.636387,3000000


In [56]:
print(df_june['mortgage_urls'].iloc[621])

https://crs.cookcountyclerkil.gov/Document/Detail?dId=MjYwOTk4MjE1&hId=YzIwZjJkYmEyZDNhZWE0YTU4OTc1YjQ5OTQ0YmYwMmJkNDVmNGY3NjM5NjQ1NmYxNTM5MjYwZThiZDBiYWU0MQ2


In [104]:
print(f'July: ${df_july.mortgage_amount_int.sum():,}')
print('----------')
print(f'August: ${df_august.mortgage_amount_int.sum():,}')
print('----------')
print(f'Total: ${df_july.mortgage_amount_int.sum() + df_august.mortgage_amount_int.sum():,}')

July: $217,098,578
----------
August: $156,032,501
----------
Total: $373,131,079


In [106]:
## US BANK july 2024
40+29+23

92

In [108]:
## US BANK August 2024
35 + 29 + 22 + 8

94

In [107]:
df_august['1st Grantor'].value_counts().head(60)

1st Grantor
NEWREZ LLC                                36
US BK NATIONAL ASSOCIATION                35
NATIONSTAR MTG LLC                        30
US BANK TRUST NATIONAL ASSN TR            29
LAKEVIEW LOAN SERVICING LLC               28
MIDFIRST BANK                             25
PENNYMAC LOAN SERVICES LLC                25
FREEDOM MTG CORPORATION                   22
US BANK NATIONAL ASSN TR                  22
WELLS FARGO BK NA                         20
JPMORGAN CHASE BK NATIONAL ASSOCIATION    19
WILMINGTON SAV FUND SOCIETY FSB TR        16
FEDERAL HOME LOAN MORTGAGE CORP TR        16
DEUTSCHE BK NATIONAL TRUST COMPANY TR     14
CARRINGTON MTG SERVICES LLC               13
FIFTH THIRD BK NATIONAL ASSOCIATION       12
PHH MTG CORPORATION                        9
WINTRUST MORTGAGE                          9
US BANK TRUST COMPANY NATIONAL ASSN TR     8
HUNTINGTON NATL BANK                       7
CITIMORTGAGE INC                           7
WELLS FARGO BK NA TR                       

In [94]:
df_august['1st Grantor'].value_counts().head(60)

1st Grantor
NEWREZ LLC                                36
US BK NATIONAL ASSOCIATION                35
NATIONSTAR MTG LLC                        30
US BANK TRUST NATIONAL ASSN TR            29
LAKEVIEW LOAN SERVICING LLC               28
MIDFIRST BANK                             25
PENNYMAC LOAN SERVICES LLC                25
FREEDOM MTG CORPORATION                   22
US BANK NATIONAL ASSN TR                  22
WELLS FARGO BK NA                         20
JPMORGAN CHASE BK NATIONAL ASSOCIATION    19
WILMINGTON SAV FUND SOCIETY FSB TR        16
FEDERAL HOME LOAN MORTGAGE CORP TR        16
DEUTSCHE BK NATIONAL TRUST COMPANY TR     14
CARRINGTON MTG SERVICES LLC               13
FIFTH THIRD BK NATIONAL ASSOCIATION       12
PHH MTG CORPORATION                        9
WINTRUST MORTGAGE                          9
US BANK TRUST COMPANY NATIONAL ASSN TR     8
HUNTINGTON NATL BANK                       7
CITIMORTGAGE INC                           7
WELLS FARGO BK NA TR                       

In [95]:
df.to_csv(f'{current_month}_JulyAugust_2024_foreclosures.csv')

In [44]:
# apr.sort_values(by='mortgage_amount_int',ascending=False)

In [96]:
print(f"July median mortgage foreclosure amount: ${df_july['mortgage_amount_int'].median():,}")

July median mortgage foreclosure amount: $178,703.0


In [97]:
print(f"August median mortgage foreclosure amount: ${df_august['mortgage_amount_int'].median():,}")

August median mortgage foreclosure amount: $171,830.0
